In [10]:
from sqlalchemy import create_engine,text
import sqlalchemy
from urllib.parse import quote
import pandas as pd

In [5]:
stock_names = sorted([
    "AARTIIND", "ABB", "ABBOTINDIA", "ABCAPITAL", "ABFRL", "ACC", "ADANIENT", "ADANIPORTS", 
    "ALKEM", "AMBUJACEM", "APOLLOHOSP", "APOLLOTYRE", "ASHOKLEY", "ASIANPAINT", "ASTRAL", 
    "ATUL", "AUBANK", "AUROPHARMA", "AXISBANK", "BAJAJ-AUTO", "BAJAJFINSV", "BAJFINANCE", 
    "BALKRISIND", "BALRAMCHIN", "BANDHANBNK", "BANKBARODA", "BATAINDIA", "BEL", "BERGEPAINT", 
    "BHARATFORG", "BHARTIARTL", "BHEL", "BIOCON", "BOSCHLTD", "BPCL", "BRITANNIA", "BSOFT", 
    "CANBK", "CANFINHOME", "CHAMBLFERT", "CHOLAFIN", "CIPLA", "COALINDIA", "COFORGE", "COLPAL", 
    "CONCOR", "COROMANDEL", "CROMPTON", "CUB", "CUMMINSIND", "DABUR", "DALBHARAT", "DEEPAKNTR", 
    "DIVISLAB", "DIXON", "DLF", "DRREDDY", "EICHERMOT", "ESCORTS", "EXIDEIND", "FEDERALBNK", 
    "GAIL", "GLENMARK", "GMRINFRA", "GNFC", "GODREJCP", "GODREJPROP", "GRANULES", "GRASIM", 
    "GUJGASLTD", "HAL", "HAVELLS", "HCLTECH", "HDFCAMC", "HDFCBANK", "HDFCLIFE", "HEROMOTOCO", 
    "HINDALCO", "HINDCOPPER", "HINDPETRO", "HINDUNILVR", "ICICIBANK", "ICICIGI", "ICICIPRULI", 
    "IDEA", "IDFC", "IDFCFIRSTB", "IEX", "IGL", "INDHOTEL", "INDIACEM", "INDIAMART", "INDIGO", 
    "INDUSINDBK", "INDUSTOWER", "INFY", "IOC", "IPCALAB", "IRCTC", "ITC", "JINDALSTEL", 
    "JKCEMENT", "JSWSTEEL", "JUBLFOOD", "KOTAKBANK", "LALPATHLAB", "LAURUSLABS", "LICHSGFIN", 
    "LT", "LTF", "LTIM", "LTTS", "LUPIN", "M&M", "M&MFIN", "MANAPPURAM", "MARICO", "MARUTI", 
    "MCX", "METROPOLIS", "MFSL", "MGL", "MOTHERSON", "MPHASIS", "MRF", 
    "MUTHOOTFIN", "NATIONALUM", "NAUKRI", "NAVINFLUOR", "NESTLEIND", "NMDC", "NTPC", 
    "OBEROIRLTY", "OFSS", "ONGC", "PAGEIND", "PEL", "PERSISTENT", "PETRONET", "PFC", 
    "PIDILITIND", "PIIND", "PNB", "POLYCAB", "POWERGRID", "PVRINOX", "RAMCOCEM", "RBLBANK", 
    "RECLTD", "RELIANCE", "SAIL", "SBICARD", "SBILIFE", "SBIN", "SHREECEM", "SHRIRAMFIN", 
    "SIEMENS", "SRF", "SUNPHARMA", "SUNTV", "SYNGENE", "TATACHEM", "TATACOMM", "TATACONSUM", 
    "TATAMOTORS", "TATAPOWER", "TATASTEEL", "TCS", "TECHM", "TITAN", "TORNTPHARM", "TRENT", 
    "TVSMOTOR", "UBL", "ULTRACEMCO", "UNITDSPR", "UPL", "VEDL", "VOLTAS", "WIPRO", 
    "ZYDUSLIFE"], key=len, reverse=True)

expiry_dates = ['2024-07-25']

engine = create_engine(
        "postgresql+psycopg2://sd_admin:%s@192.168.1.72:5430/stock-dumps"
        % quote("sdadmin@postgres")
    )

In [6]:
# create stocks df
# name nifty_fifty_index
df = pd.DataFrame(
        {'name':stock_names, 'nifty_fifty_index':0}
    )
print(df)

           name  nifty_fifty_index
0    ABBOTINDIA                  0
1    ADANIPORTS                  0
2    APOLLOHOSP                  0
3    APOLLOTYRE                  0
4    ASIANPAINT                  0
..          ...                ...
176         SRF                  0
177         TCS                  0
178         UBL                  0
179         UPL                  0
180          LT                  0

[181 rows x 2 columns]


In [8]:
# insert into stock table
df.to_sql("stock", schema="options", if_exists="append", con=engine, index=False)

181

In [11]:
# create expiry df
# date
expiry_df = pd.DataFrame({'date':expiry_dates})

In [12]:
expiry_df.to_sql('expiry', schema="options", if_exists="append", con=engine, index=False)

1

In [12]:
# read expiry, stock to a df
with engine.connect() as connection:
    result = connection.execute(text("SELECT * FROM options.stock"))
    rows = result.fetchall()
    columns = result.keys()
df = pd.DataFrame(rows, columns=columns)
print(df)

# create ticker df 
# symbol_token stock_id expiry_id trade_date strike_price option_type

# ticker_df = 

     stock_id        name  nifty_fifty_index
0           1  ABBOTINDIA                  0
1           2  ADANIPORTS                  0
2           3  APOLLOHOSP                  0
3           4  APOLLOTYRE                  0
4           5  ASIANPAINT                  0
..        ...         ...                ...
176       177         SRF                  0
177       178         TCS                  0
178       179         UBL                  0
179       180         UPL                  0
180       181          LT                  0

[181 rows x 3 columns]
